In [1]:
from bigraph_schema.units import units, parse_coefficient, parse_dimensionality, render_coefficient, render_units_type
from ecoli.shared.registry import ecoli_core as ec
from ecoli.shared.utils.schemas import get_defaults_schema
from ecoli.migrated.antibiotics.cell_wall import CellWall
from ecoli.shared.interface import ProcessBase
from vivarium import Vivarium
from ecoli.shared.utils.schemas import get_config_schema, collapse_defaults, get_defaults_schema
from ecoli.migrated.antibiotics.death import ToyDeath

ec.process_registry.register('cell-wall', CellWall)
viv = Vivarium(core=ec, types=ec.types())
# viv.process_interface('ecoli-cell-wall')

<unknown>:1: SyntaxWarning: invalid escape sequence '\s'


In [15]:
def infer_state_from_composer(composer):
    composition = composer.generate()
    processes = composition.get('processes')
    topology = composition.get('topology')
    
    state = {}
    for process_id, process in processes.items():
        ports = topology.get(process_id)
        state[process_id] = {
            "_type": "process",
            "address": f"local:{process_id}",
            "config": {},  # get config from process
            "inputs": ports,
            "outputs": ports
        }
    return state


composer = ToyDeath({})
state = infer_state_composition(composer)
state

{'death': {'_type': 'process',
  'address': 'local:death',
  'inputs': {'internal': ('cell',), 'global': ('global',), 'processes': ()},
  'outputs': {'internal': ('cell',), 'global': ('global',), 'processes': ()}},
 'injector': {'_type': 'process',
  'address': 'local:injector',
  'inputs': {'internal': ('cell',)},
  'outputs': {'internal': ('cell',)}},
 'enduring_injector': {'_type': 'process',
  'address': 'local:enduring_injector',
  'inputs': {'internal': ('cell',)},
  'outputs': {'internal': ('cell',)}}}

In [2]:
# defining a SubClass 
class Cell(ProcessBase): 
    defaults = {
        # Molecules
        "murein": "CPD-12261[p]",  # four crosslinked peptidoglycan units
        "PBP": {  # penicillin-binding proteins
            "PBP1A": "CPLX0-7717[m]",  # transglycosylase-transpeptidase
            # PBP1B has three isoforms: α (currently not produced by model),
            # β (degradation product of α, not in vivo), and γ (made by model)
            "PBP1B_alpha": "CPLX0-3951[i]",
            "PBP1B_gamma": "CPLX0-8300[c]",
        }
    }

    def initialize(self, config):
        self.murein = config['murein']
  
subclass = Cell({}, ec) 
subclass.murein

'CPD-12261[p]'

In [5]:
subclass.timestep

1.0

In [1]:
from ecoli.shared.utils.schemas import get_config_schema, collapse_defaults, get_defaults_schema

class iX:
    defaults = {}
    config_schema = {}

    def __new__(cls, k):
        instance = super().__new__(cls)
        instance.config_schema = {
            **get_config_schema(cls.defaults),
            "time_step": {"_default": 1.0, "_type": "float"}
        }
        return instance
        
    def __init__(self, k):
        self.k = k


class X1(iX):
    defaults = {
        "murein": "CPD-12261[p]",  # four crosslinked peptidoglycan units
        "PBP": {  # penicillin-binding proteins
            "PBP1A": "CPLX0-7717[m]",  # transglycosylase-transpeptidase
            "PBP1B_alpha": "CPLX0-3951[i]",
            "PBP1B_gamma": "CPLX0-8300[c]",
        }
    }


x = X1(11.11)

x.config_schema

<unknown>:1: SyntaxWarning: invalid escape sequence '\s'


{'murein': {'_type': 'string', '_default': 'CPD-12261[p]'},
 'PBP': {'_type': 'tree',
  '_default': {'PBP1A': 'CPLX0-7717[m]',
   'PBP1B_alpha': 'CPLX0-3951[i]',
   'PBP1B_gamma': 'CPLX0-8300[c]'}},
 'time_step': {'_default': 1.0, '_type': 'float'}}

In [6]:
initial_state = {
        "antibiotic": {
            "species": {
                "periplasm": {
                    "_default": 0 * units.mM,
                    "_divider": "polynomial",
                    "_type": "unit"
                }
               
            },
        }
}


get_defaults_schema(initial_state)

{'antibiotic': {'species': {'periplasm': {'_divide': 'polynomial',
    '_type': 'unit'}}}}

In [7]:
from bigraph_schema.units import units, parse_coefficient, parse_dimensionality, render_coefficient, render_units_type
from ecoli.shared.registry import ecoli_core as ec


initial_state = {
        "antibiotic": {
            "species": {
                "periplasm": 0 * units.mM,
                "hydrolyzed_periplasm": 0 * units.mM,
                "cytoplasm": 0 * units.mM,
                "hydrolyzed_cytoplasm": 0 * units.mM,
                "external": 3 * units.mM,
            },
            "reaction_parameters": {
                "diffusion": {
                    "outer_permeability": 2 * units.dm / units.sec,
                    "outer_area": 3 * units.dm**2,
                    "periplasm_volume": 2 * units.L,
                    "charge": 0 * units.count,
                    "inner_permeability": 0 * units.dm / units.sec,
                    "inner_area": 3 * units.dm**2,
                    "cytoplasm_volume": 2 * units.L,
                },
                "export": {
                    "outer_kcat": 4 / units.sec,
                    "outer_km": 2 * units.mM,
                    "outer_enzyme_conc": 1 * units.mM,
                    "outer_n": 1 * units.count,
                    "inner_kcat": 0 / units.sec,
                    "inner_km": 2 * units.mM,
                    "inner_enzyme_conc": 1 * units.mM,
                    "inner_n": 1 * units.count,
                },
                "hydrolysis": {
                    "outer_kcat": 4 / units.sec,
                    "outer_km": 2 * units.mM,
                    "outer_enzyme_conc": 0.5 * units.mM,
                    "outer_n": 1 * units.count,
                    "inner_kcat": 0 / units.sec,
                    "inner_km": 2 * units.mM,
                    "inner_enzyme_conc": 0.5 * units.mM,
                    "inner_n": 1 * units.count,
                },
            },
        },
    }


eval("0 * units.units")

<Quantity(0, 'micronit')>

In [3]:
from vivarium.library.units import units as vivunits
from ecoli.shared.registry import ecoli_core as ec
from bigraph_schema.type_functions import apply_units, check_units, serialize_units, deserialize_units

u = 1 * vivunits.mM / vivunits.mg

serial = serialize_units({}, u, ec)
deserial = deserialize_units({}, serial, ec)
serial

'1.0 millimolar / milligram'

In [1]:
from ecoli import shared
import os 


os.path.exists()

<unknown>:1: SyntaxWarning: invalid escape sequence '\s'


True

In [39]:
dir(deserial)

['T',
 'UnitsContainer',
 '_NumpyQuantity__ito_if_needed',
 '_REGISTRY',
 '__abs__',
 '__add__',
 '__annotations__',
 '__array__',
 '__array_function__',
 '__array_priority__',
 '__array_ufunc__',
 '__bool__',
 '__bytes__',
 '__class__',
 '__class_getitem__',
 '__complex__',
 '__copy__',
 '__dask_graph__',
 '__dask_keys__',
 '__dask_optimize__',
 '__dask_postcompute__',
 '__dask_postpersist__',
 '__dask_scheduler__',
 '__dask_tokenize__',
 '__deepcopy__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__div__',
 '__divmod__',
 '__doc__',
 '__eq__',
 '__float__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__idiv__',
 '__ifloordiv__',
 '__imod__',
 '__imul__',
 '__init__',
 '__init_subclass__',
 '__int__',
 '__ipow__',
 '__isub__',
 '__iter__',
 '__itruediv__',
 '__le__',
 '__len__',
 '__lt__',
 '__matmul__',
 '__mod__',
 '__module__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '

In [9]:
bidirectional_ports = {"wall_state": {
                "lattice": {
                    "_default": None,
                    "_type": "list[list[float]]",
                    "_divider": 'd'
                },
                "cracked": {
                    "_default": False,
                    # TODO: does this need a divider?
                    # "_divider": {"divider": "set_value", "config": {"value": False}},
                },
            }
        }


input_ports = {
    "wall_state": {
        **bidirectional_ports['wall_state'],
        'x': {}
    }
}

In [11]:
input_ports['wall_state'].keys()

dict_keys(['lattice', 'cracked', 'x'])